In [1]:
import os
from typing import List, Tuple, Dict, Optional

import numpy as np
import cv2
from sklearn.decomposition import PCA
from sklearn.metrics import accuracy_score

In [2]:
class PCABasedFaceRecognition:
    def __init__(
        self,
        dataset_path: str = "FaceRecognitionDset/lfw_funneled",
        n_components: float | int = 0.95,  # keep 95% variance by default
        whiten: bool = True,
        random_state: int = 42,
        image_size: Tuple[int, int] = (100, 100),
        use_zscore: bool = True,  # keep consistent with baseline by default
    ):
        self.dataset_path = dataset_path
        self.n_components = n_components
        self.whiten = whiten
        self.random_state = random_state
        self.image_size = image_size
        self.use_zscore = use_zscore

        self.pca: Optional[PCA] = None

    def _image_path(self, person_name: str, image_num: int) -> str:
        return os.path.join(
            self.dataset_path, person_name, f"{person_name}_{image_num:04d}.jpg"
        )

    def load_image(self, image_path: str) -> Optional[np.ndarray]:
        """Load image, convert to grayscale 100x100, gaussian blur; return flattened float32 array."""
        try:
            img = cv2.imread(image_path)
            if img is None:
                return None
            img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
            img = cv2.resize(img, self.image_size)
            img = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
            img = cv2.GaussianBlur(img, (3, 3), 0)
            vec = img.flatten().astype(np.float32)
            # Normalize to [0,1]
            vec = vec / 255.0
            if self.use_zscore:
                # Per-image z-score (to mirror baseline behavior)
                mu = float(vec.mean())
                sigma = float(vec.std()) + 1e-8
                vec = (vec - mu) / sigma
            return vec
        except Exception:
            return None

    def extract_features(self, person_name: str, image_num: int) -> Optional[np.ndarray]:
        path = self._image_path(person_name, image_num)
        return self.load_image(path)

    def fit_pca_from_pairs(
        self, pairs_file: str, max_images: int = 800
    ) -> Tuple[int, int]:
        """
        Fit PCA using features of images referenced in the given pairs file.
        We collect unique images until `max_images` reached (or file exhausted).
        Returns (n_samples, n_features_before).
        """
        # Collect unique (person, img_num) references from pairs
        with open(pairs_file, "r") as f:
            lines = [ln.strip() for ln in f.readlines()]
        if not lines:
            raise RuntimeError("Pairs file is empty")

        # Skip first header line (may contain counts)
        unique_refs: set[Tuple[str, int]] = set()
        for line in lines[1:]:
            if not line:
                continue
            parts = line.split()
            if len(parts) == 3:
                person, img1, img2 = parts
                unique_refs.add((person, int(img1)))
                unique_refs.add((person, int(img2)))
            elif len(parts) == 4:
                person1, img1, person2, img2 = parts
                unique_refs.add((person1, int(img1)))
                unique_refs.add((person2, int(img2)))
            if len(unique_refs) >= max_images:
                break

        # Load features
        X: List[np.ndarray] = []
        kept = 0
        for p, i in unique_refs:
            feat = self.extract_features(p, i)
            if feat is not None:
                X.append(feat)
                kept += 1
        if not X:
            raise RuntimeError("No features loaded to fit PCA")

        X_mat = np.vstack(X)  # shape (n_samples, D)
        # Fit PCA
        self.pca = PCA(n_components=self.n_components, whiten=self.whiten, random_state=self.random_state)
        self.pca.fit(X_mat)
        return X_mat.shape

    def transform(self, features: Optional[np.ndarray]) -> Optional[np.ndarray]:
        if features is None:
            return None
        if self.pca is None:
            raise RuntimeError("PCA has not been fit. Call fit_pca_from_pairs first.")
        return self.pca.transform(features.reshape(1, -1)).ravel()

    @staticmethod
    def euclidean_distance(v1: Optional[np.ndarray], v2: Optional[np.ndarray]) -> float:
        if v1 is None or v2 is None:
            return float("inf")
        return float(np.linalg.norm(v1 - v2))

    def verify_pair(self, person1: str, img1: int, person2: str, img2: int, threshold: float) -> bool:
        f1 = self.extract_features(person1, img1)
        f2 = self.extract_features(person2, img2)
        if f1 is None or f2 is None:
            return False
        tf1 = self.transform(f1)
        tf2 = self.transform(f2)
        d = self.euclidean_distance(tf1, tf2)
        return d < threshold

    def _collect_distances(self, pairs_file: str, max_pairs: int = 200) -> Tuple[List[float], List[float]]:
        with open(pairs_file, "r") as f:
            lines = f.readlines()
        if not lines:
            return [], []

        distances_same: List[float] = []
        distances_diff: List[float] = []
        same_count = diff_count = 0
        line_idx = 1  # skip header
        while line_idx < len(lines) and (same_count < max_pairs // 2 or diff_count < max_pairs // 2):
            parts = lines[line_idx].strip().split()
            if len(parts) == 3 and same_count < max_pairs // 2:
                person, img1, img2 = parts
                f1 = self.extract_features(person, int(img1))
                f2 = self.extract_features(person, int(img2))
                if f1 is not None and f2 is not None:
                    tf1 = self.transform(f1)
                    tf2 = self.transform(f2)
                    distances_same.append(self.euclidean_distance(tf1, tf2))
                    same_count += 1
            elif len(parts) == 4 and diff_count < max_pairs // 2:
                person1, img1, person2, img2 = parts
                f1 = self.extract_features(person1, int(img1))
                f2 = self.extract_features(person2, int(img2))
                if f1 is not None and f2 is not None:
                    tf1 = self.transform(f1)
                    tf2 = self.transform(f2)
                    distances_diff.append(self.euclidean_distance(tf1, tf2))
                    diff_count += 1
            line_idx += 1
        return distances_same, distances_diff

    def find_optimal_threshold(self, pairs_file: str, max_pairs: int = 200) -> float:
        ds, dd = self._collect_distances(pairs_file, max_pairs=max_pairs)
        if not ds or not dd:
            raise RuntimeError("Could not compute distances for threshold search")
        min_dist = min(min(ds), min(dd))
        max_dist = max(max(ds), max(dd))
        thresholds = np.linspace(min_dist * 0.8, max_dist * 1.2, 100)
        best_acc = -1.0
        best_thr = float(np.mean([np.mean(ds), np.mean(dd)]))
        for thr in thresholds:
            correct = sum(d < thr for d in ds) + sum(d >= thr for d in dd)
            total = len(ds) + len(dd)
            acc = correct / total if total > 0 else 0.0
            if acc > best_acc:
                best_acc = acc
                best_thr = thr
        print(f"Processed {len(ds)} same-person pairs, {len(dd)} different-person pairs")
        print(f"Same person distances: mean={np.mean(ds):.3f}, std={np.std(ds):.3f}")
        print(f"Different person distances: mean={np.mean(dd):.3f}, std={np.std(dd):.3f}")
        print(f"Optimal threshold: {best_thr:.3f} (accuracy: {best_acc:.3f})")
        return float(best_thr)

In [3]:
def evaluate_face_recognition(recognizer: PCABasedFaceRecognition, pairs_file: str, threshold: float, max_pairs: int = 400):
    with open(pairs_file, "r") as f:
        lines = f.readlines()
    if not lines:
        raise RuntimeError("Empty pairs file")

    predictions: List[bool] = []
    ground_truth: List[bool] = []

    same_count = diff_count = 0
    line_idx = 1
    while line_idx < len(lines) and (same_count < max_pairs // 2 or diff_count < max_pairs // 2):
        parts = lines[line_idx].strip().split()
        if len(parts) == 3 and same_count < max_pairs // 2:
            person, img1, img2 = parts
            pred = recognizer.verify_pair(person, int(img1), person, int(img2), threshold)
            predictions.append(pred)
            ground_truth.append(True)
            same_count += 1
        elif len(parts) == 4 and diff_count < max_pairs // 2:
            person1, img1, person2, img2 = parts
            pred = recognizer.verify_pair(person1, int(img1), person2, int(img2), threshold)
            predictions.append(pred)
            ground_truth.append(False)
            diff_count += 1
        line_idx += 1

    acc = accuracy_score(ground_truth, predictions)
    tp = sum(p and gt for p, gt in zip(predictions, ground_truth))
    fp = sum(p and not gt for p, gt in zip(predictions, ground_truth))
    fn = sum((not p) and gt for p, gt in zip(predictions, ground_truth))
    tn = sum((not p) and (not gt) for p, gt in zip(predictions, ground_truth))

    precision = tp / (tp + fp) if (tp + fp) > 0 else 0.0
    recall = tp / (tp + fn) if (tp + fn) > 0 else 0.0
    f1 = (2 * precision * recall / (precision + recall)) if (precision + recall) > 0 else 0.0

    print("Evaluation Results:")
    print(f"Tested {same_count} same-person pairs, {diff_count} different-person pairs")
    print(f"Accuracy: {acc:.3f}")
    print(f"Precision: {precision:.3f}")
    print(f"Recall: {recall:.3f}")
    print(f"F1-Score: {f1:.3f}")
    print(f"True Positives: {tp}, False Positives: {fp}")
    print(f"True Negatives: {tn}, False Negatives: {fn}")
    return acc, precision, recall, f1

In [4]:
recognizer = PCABasedFaceRecognition(
    dataset_path="FaceRecognitionDset/lfw_funneled",
    n_components=0.95,  # keep 95% variance
    whiten=False,
    random_state=42,
    image_size=(100, 100),
    use_zscore=False,
)

# 1) Fit PCA using dev-train referenced images
print("Fitting PCA from development train pairs...")
try:
    n_samples, n_features = recognizer.fit_pca_from_pairs(
        "FaceRecognitionDset/pairsDevTrain.txt", max_images=800
    )
    print(f"PCA fitted with {n_samples} samples, original dim={n_features}, reduced dim={recognizer.pca.n_components_ if recognizer.pca is not None else 'N/A'}")
except Exception as e:
    print(f"Error during PCA fitting: {e}")
    raise

# 2) Find optimal threshold on dev-train distances (or dev-test)
print("\nSearching optimal threshold on dev-train distances...")
optimal_threshold = recognizer.find_optimal_threshold(
    "FaceRecognitionDset/pairsDevTrain.txt", max_pairs=200
)

# 3) Evaluate
print("\nTesting on development test set:")
evaluate_face_recognition(
    recognizer, "FaceRecognitionDset/pairsDevTest.txt", optimal_threshold, max_pairs=200
)

print("\nTesting on full evaluation set:")
evaluate_face_recognition(
    recognizer, "FaceRecognitionDset/pairs.txt", optimal_threshold, max_pairs=400
)

Fitting PCA from development train pairs...
PCA fitted with 800 samples, original dim=10000, reduced dim=159

Searching optimal threshold on dev-train distances...


/Users/derek/.pyenv/versions/3.12.11/lib/python3.12/site-packages/sklearn/decomposition/_base.py:148: RuntimeWarning: divide by zero encountered in matmul
  X_transformed = X @ self.components_.T
/Users/derek/.pyenv/versions/3.12.11/lib/python3.12/site-packages/sklearn/decomposition/_base.py:148: RuntimeWarning: overflow encountered in matmul
  X_transformed = X @ self.components_.T
/Users/derek/.pyenv/versions/3.12.11/lib/python3.12/site-packages/sklearn/decomposition/_base.py:148: RuntimeWarning: invalid value encountered in matmul
  X_transformed = X @ self.components_.T
/Users/derek/.pyenv/versions/3.12.11/lib/python3.12/site-packages/sklearn/decomposition/_base.py:155: RuntimeWarning: divide by zero encountered in matmul
  X_transformed -= xp.reshape(self.mean_, (1, -1)) @ self.components_.T
/Users/derek/.pyenv/versions/3.12.11/lib/python3.12/site-packages/sklearn/decomposition/_base.py:155: RuntimeWarning: overflow encountered in matmul
  X_transformed -= xp.reshape(self.mean_, (

Processed 100 same-person pairs, 100 different-person pairs
Same person distances: mean=31.461, std=8.948
Different person distances: mean=32.597, std=8.012
Optimal threshold: 29.192 (accuracy: 0.560)

Testing on development test set:


/Users/derek/.pyenv/versions/3.12.11/lib/python3.12/site-packages/sklearn/decomposition/_base.py:148: RuntimeWarning: divide by zero encountered in matmul
  X_transformed = X @ self.components_.T
/Users/derek/.pyenv/versions/3.12.11/lib/python3.12/site-packages/sklearn/decomposition/_base.py:148: RuntimeWarning: overflow encountered in matmul
  X_transformed = X @ self.components_.T
/Users/derek/.pyenv/versions/3.12.11/lib/python3.12/site-packages/sklearn/decomposition/_base.py:148: RuntimeWarning: invalid value encountered in matmul
  X_transformed = X @ self.components_.T
/Users/derek/.pyenv/versions/3.12.11/lib/python3.12/site-packages/sklearn/decomposition/_base.py:155: RuntimeWarning: divide by zero encountered in matmul
  X_transformed -= xp.reshape(self.mean_, (1, -1)) @ self.components_.T
/Users/derek/.pyenv/versions/3.12.11/lib/python3.12/site-packages/sklearn/decomposition/_base.py:155: RuntimeWarning: overflow encountered in matmul
  X_transformed -= xp.reshape(self.mean_, (

Evaluation Results:
Tested 100 same-person pairs, 100 different-person pairs
Accuracy: 0.590
Precision: 0.610
Recall: 0.500
F1-Score: 0.549
True Positives: 50, False Positives: 32
True Negatives: 68, False Negatives: 50

Testing on full evaluation set:


/Users/derek/.pyenv/versions/3.12.11/lib/python3.12/site-packages/sklearn/decomposition/_base.py:148: RuntimeWarning: divide by zero encountered in matmul
  X_transformed = X @ self.components_.T
/Users/derek/.pyenv/versions/3.12.11/lib/python3.12/site-packages/sklearn/decomposition/_base.py:148: RuntimeWarning: overflow encountered in matmul
  X_transformed = X @ self.components_.T
/Users/derek/.pyenv/versions/3.12.11/lib/python3.12/site-packages/sklearn/decomposition/_base.py:148: RuntimeWarning: invalid value encountered in matmul
  X_transformed = X @ self.components_.T
/Users/derek/.pyenv/versions/3.12.11/lib/python3.12/site-packages/sklearn/decomposition/_base.py:155: RuntimeWarning: divide by zero encountered in matmul
  X_transformed -= xp.reshape(self.mean_, (1, -1)) @ self.components_.T
/Users/derek/.pyenv/versions/3.12.11/lib/python3.12/site-packages/sklearn/decomposition/_base.py:155: RuntimeWarning: overflow encountered in matmul
  X_transformed -= xp.reshape(self.mean_, (

Evaluation Results:
Tested 200 same-person pairs, 200 different-person pairs
Accuracy: 0.570
Precision: 0.571
Recall: 0.565
F1-Score: 0.568
True Positives: 113, False Positives: 85
True Negatives: 115, False Negatives: 87


/Users/derek/.pyenv/versions/3.12.11/lib/python3.12/site-packages/sklearn/decomposition/_base.py:148: RuntimeWarning: divide by zero encountered in matmul
  X_transformed = X @ self.components_.T
/Users/derek/.pyenv/versions/3.12.11/lib/python3.12/site-packages/sklearn/decomposition/_base.py:148: RuntimeWarning: overflow encountered in matmul
  X_transformed = X @ self.components_.T
/Users/derek/.pyenv/versions/3.12.11/lib/python3.12/site-packages/sklearn/decomposition/_base.py:148: RuntimeWarning: invalid value encountered in matmul
  X_transformed = X @ self.components_.T
/Users/derek/.pyenv/versions/3.12.11/lib/python3.12/site-packages/sklearn/decomposition/_base.py:155: RuntimeWarning: divide by zero encountered in matmul
  X_transformed -= xp.reshape(self.mean_, (1, -1)) @ self.components_.T
/Users/derek/.pyenv/versions/3.12.11/lib/python3.12/site-packages/sklearn/decomposition/_base.py:155: RuntimeWarning: overflow encountered in matmul
  X_transformed -= xp.reshape(self.mean_, (

(0.57, 0.5707070707070707, 0.565, 0.5678391959798995)